# DataLoader Setup

In [1]:
import os
import torch
import torchvision
from pytorch_lightning import seed_everything
from pl_bolts.datamodules import CIFAR10DataModule, TinyCIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization

PATH_DATASETS = os.environ.get('PATH_DATASETS', '../datasets')
AVAIL_GPUS = min(1, torch.cuda.device_count())
BATCH_SIZE = 512 if AVAIL_GPUS else 64
NUM_WORKERS = int(os.cpu_count()/2)
TEST_ONLY = False

test_transforms = [
        torchvision.transforms.ToTensor(),
        cifar10_normalization(),        
        torchvision.transforms.Lambda(lambda x: x.clamp(min = -1, max = 1)),
        torchvision.transforms.Lambda(lambda x: x * 127),   
        torchvision.transforms.Lambda(lambda x: x.floor())  
]

train_transforms = torchvision.transforms.Compose(
    [
        torchvision.transforms.RandomCrop(32, padding=4),
        torchvision.transforms.RandomHorizontalFlip(),
    ] 
    + test_transforms
)

test_transforms = torchvision.transforms.Compose(test_transforms)

cifar10_dm = CIFAR10DataModule(
    data_dir = PATH_DATASETS,
    batch_size = BATCH_SIZE,
    num_workers = NUM_WORKERS,
    train_transforms = train_transforms,
    test_transforms = test_transforms,
    val_transforms = test_transforms,
)

seed_everything(seed=1234, workers=True)

/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:73: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:77: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:81: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
Global seed set to 1234


1234

# model 

In [2]:
from model import ReactModel

baseline_real = [{'conv':'real','in_channels':3, 'out_channels':32, 'stride':1}, # shpae (-1,32,32,32)
             # first layer
             {'conv':'scaled_sign','in_channels':32, 'out_channels':64, 'stride':2, 'kernel_size':3, 'padding':1}, # output shape (-1,64,16,16), Reduction
             {'conv':'scaled_sign','in_channels':64, 'out_channels':128, 'stride':2, 'kernel_size':3, 'padding':1}, # output shape (-1,128,8,8), Reduction
             {'conv':'scaled_sign','in_channels':128, 'out_channels':128, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,128,8,8), Normal
             {'conv':'scaled_sign','in_channels':128, 'out_channels':256, 'stride':2, 'kernel_size':3, 'padding':1}, # output shape (-1,256,4,4), Reduction
             {'conv':'scaled_sign','in_channels':256, 'out_channels':256, 'stride':1, 'kernel_size':3, 'padding':1}, # output shape (-1,256,4,4), Normal
             {'conv':'scaled_sign','in_channels':256, 'out_channels':512, 'stride':2, 'kernel_size':3, 'padding':1}, # output shape (-1,512,2,2), Reduction
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':1, 'padding':0}, # output shape (-1,512,2,2), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':1, 'padding':0}, # output shape (-1,512,2,2), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':1, 'padding':0}, # output shape (-1,512,2,2), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':1, 'padding':0}, # output shape (-1,512,2,2), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':512, 'stride':1, 'kernel_size':1, 'padding':0}, # output shape (-1,512,2,2), Normal
             {'conv':'scaled_sign','in_channels':512, 'out_channels':1024, 'stride':2, 'kernel_size':1, 'padding':0}, # output shape (-1,512,1,1), Reduction
             {'conv':'fc','in_channels':1024, 'out_channels':1024, 'stride':1, 'kernel_size':1, 'padding':0,'dropout':0.5}, # output shape (-1,1024,1,1), Normal
             {'conv':'fc','in_channels':1024, 'out_channels':10, 'stride':1, 'kernel_size':1, 'padding':0,'dropout':0.5}] # output shape (-1,10,1,1)

baseline_model = ReactModel(
    structure=baseline_real,
    adam_init_lr=0.01, 
    adam_weight_decay=0,
    adam_betas=(0.9, 0.999),
    lr_reduce_factor=0.1,
    lr_patience=50,
    limit_conv_weight=True,
    limit_bn_weight=True,
    )

print(baseline_model.hparams)

"adam_betas":        (0.9, 0.999)
"adam_init_lr":      0.01
"adam_weight_decay": 0
"limit_bn_weight":   True
"limit_conv_weight": True
"lr_patience":       50
"lr_reduce_factor":  0.1
"structure":         [{'conv': 'real', 'in_channels': 3, 'out_channels': 32, 'stride': 1}, {'conv': 'scaled_sign', 'in_channels': 32, 'out_channels': 64, 'stride': 2, 'kernel_size': 3, 'padding': 1}, {'conv': 'scaled_sign', 'in_channels': 64, 'out_channels': 128, 'stride': 2, 'kernel_size': 3, 'padding': 1}, {'conv': 'scaled_sign', 'in_channels': 128, 'out_channels': 128, 'stride': 1, 'kernel_size': 3, 'padding': 1}, {'conv': 'scaled_sign', 'in_channels': 128, 'out_channels': 256, 'stride': 2, 'kernel_size': 3, 'padding': 1}, {'conv': 'scaled_sign', 'in_channels': 256, 'out_channels': 256, 'stride': 1, 'kernel_size': 3, 'padding': 1}, {'conv': 'scaled_sign', 'in_channels': 256, 'out_channels': 512, 'stride': 2, 'kernel_size': 3, 'padding': 1}, {'conv': 'scaled_sign', 'in_channels': 512, 'out_channels': 51

In [3]:
print(baseline_model)



ReactModel(
  (blocks): ModuleList(
    (0): firstconv3x3(
      (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): DWConvReact(32, 64, conv=scaled_sign, block=Reduction)
    (2): DWConvReact(64, 128, conv=scaled_sign, block=Reduction)
    (3): DWConvReact(128, 128, conv=scaled_sign, block=Normal)
    (4): DWConvReact(128, 256, conv=scaled_sign, block=Reduction)
    (5): DWConvReact(256, 256, conv=scaled_sign, block=Normal)
    (6): DWConvReact(256, 512, conv=scaled_sign, block=Reduction)
    (7): DWConvReact(512, 512, conv=scaled_sign, block=Normal)
    (8): DWConvReact(512, 512, conv=scaled_sign, block=Normal)
    (9): DWConvReact(512, 512, conv=scaled_sign, block=Normal)
    (10): DWConvReact(512, 512, conv=scaled_sign, block=Normal)
    (11): DWConvReact(512, 512, conv=scaled_sign, block=Normal)
    (12): DWConvReact(512

In [4]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_callback = ModelCheckpoint(filename='{epoch}-{val_loss:.4f}-{val_acc:.4f}', monitor='val_acc', mode='max')
trainer = Trainer(
    max_epochs=-1,
    gpus=AVAIL_GPUS,
    logger=TensorBoardLogger('lightning_logs/', name='no_teacher_binary_react', log_graph=True),
    # logger=TensorBoardLogger('lightning_logs/', name='Real', log_graph=True),
    callbacks=[LearningRateMonitor(logging_interval='step'), 
               EarlyStopping(monitor='val_acc', mode='max', patience=100),
               checkpoint_callback],
    deterministic=True,
#    gradient_clip_val = 0.5
)

trainer.fit(baseline_model, datamodule=cifar10_dm)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Files already downloaded and verified
Files already downloaded and verified


/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:114: LightningDeprecationWarning: DataModule property `train_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:133: LightningDeprecationWarning: DataModule property `val_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/loggers/tensorboard.py:247: UserWarning: Could not log computational graph since the `model.example_input_array` attribute is not set or `input_array` was not given
  rank_zero_warn(

  | Name   | Type       | Params
--------------------------------------
0 | blocks | ModuleList | 6.0 M 
--------------------------------------
6.0 M     Trainable params
0         Non-trainab

Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 1234


...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [5]:
trainer.test(baseline_model,cifar10_dm)

/home/neungyun/anaconda3/envs/torch/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:152: LightningDeprecationWarning: DataModule property `test_transforms` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8366000056266785, 'test_loss': 0.5316360592842102}
--------------------------------------------------------------------------------


[{'test_loss': 0.5316360592842102, 'test_acc': 0.8366000056266785}]